<a href="https://colab.research.google.com/github/michsund/DATA612_Assignments/blob/main/Sunderman_Assignment_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment for Week 7

## Purpose          
This assignment is intended to help you do the following:      
  - Groupby operations to aggregate, transform, and filter data      
  - Built-in and custom user functions to perform groupby operations


## About the assignment

The function 'get_data_table' takes 4 parameters, df, date_col, group_col, type_x.      
  - The 'df' represents the dataframe.      
  -The 'date_col' represents the date column.      
  -The group_col represents the column to groupby and        
  -The 'type_x' represents the column with a specific value/category == 'type_x'. 

## Tasks   
  1. Read the function above and in a 1-3 paragraphs explain its purpose    

  2. Apply the function to your selected dataframe and modify as needed based on your preferences.    

  3. Write 1-3 paragraphs to explain the differences or challenges when you applied the function to your chosen dataframe.

  4. Post your version of the above code on your GitHub account. Do not forget a README file with explanations.




The goal of the function is to input the data frame, a column date, a variable to group by, and a category to filter by.  When entering the appropriate variables, the df_gross function should yield some total amount as specified in the ['amount'] that is summed.  The df_recovered function should yield another specified variable that is summed for a total amount.  The return function then adds the gross and returned function that is multiplied by -1, thereby subtracting it from the gross total.  The function combines this resulting difference, grouped by the date and category selected.

I chose to work with a different data set for this exercise, using Columbus, OH AirBnB calendar data found at http://insideairbnb.com/get-the-data.html. 

In [482]:
import pandas as pd
import numpy as np

In [483]:
df = pd.read_csv('http://data.insideairbnb.com/united-states/oh/columbus/2020-10-25/data/calendar.csv.gz')

In [484]:
df.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,13446477,2020-10-26,t,$33.00,$33.00,1.0,1125.0
1,90676,2020-10-25,f,$166.00,$166.00,1.0,365.0
2,90676,2020-10-26,t,$80.00,$80.00,1.0,365.0
3,90676,2020-10-27,t,$80.00,$80.00,1.0,365.0
4,90676,2020-10-28,t,$80.00,$80.00,1.0,365.0


In [485]:
df['price'] = df['price'].str.replace('$', '')
df['price'] = df['price'].str.replace('.00', '')
df['price'] = df['price'].str.replace(',', '')
df['price'] = pd.to_numeric(df['price'])

In [486]:
df['adjusted_price'] = df['adjusted_price'].str.replace('$', '')
df['adjusted_price'] = df['adjusted_price'].str.replace('.00', '')
df['adjusted_price'] = df['adjusted_price'].str.replace(',', '')
df['adjusted_price'] = pd.to_numeric(df['adjusted_price'])

In [487]:
df['month'] = pd.to_datetime(df.date).dt.to_period('M')

In [488]:
df.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights,month
0,13446477,2020-10-26,t,33.0,33.0,1.0,1125.0,2020-10
1,90676,2020-10-25,f,166.0,166.0,1.0,365.0,2020-10
2,90676,2020-10-26,t,80.0,80.0,1.0,365.0,2020-10
3,90676,2020-10-27,t,80.0,80.0,1.0,365.0,2020-10
4,90676,2020-10-28,t,80.0,80.0,1.0,365.0,2020-10


In [489]:
df.groupby(['date', 'listing_id']) [['price']].sum()

price
date       listing_id       
2020-10-25 90676       166.0
           591101       80.0
           681145      115.0
           923248       34.0
           927867       70.0
...                      ...
2021-10-25 45955138     70.0
           45959893     41.0
           45980528     69.0
           45984196     44.0
           46013461     68.0

[422310 rows x 1 columns]

In [490]:
df.month = df.month.astype(str)

In [509]:
def get_data_table(df, date_col, group_col, type_x):
    df_gross = (df.loc[(df.available == type_x) & df['month'].isin(['2020-10', '2020-11', '2020-12'])].groupby([group_col, date_col]).sum().unstack(group_col)['price'].fillna(0).resample('D', axis=1).sum())

    df_recovered = (df.loc[(df.available == type_x) & df['month'].isin(['2020-10', '2020-11', '2020-12'])].groupby([group_col, date_col]).sum().unstack(group_col)['adjusted_price'].fillna(0).resample('D', axis=1).sum())

    return df_gross.add(df_recovered * -1, fill_value=0)
    

In [510]:
attempt = get_data_table(df, df['date'], df['listing_id'],'t')

ValueError: ignored

In [511]:
df['pricedif'] = df['price'] + (df['adjusted_price'] * -1)

In [512]:
df.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights,month,pricedif
0,13446477,2020-10-26,t,33.0,33.0,1.0,1125.0,2020-10,0.0
1,90676,2020-10-25,f,166.0,166.0,1.0,365.0,2020-10,0.0
2,90676,2020-10-26,t,80.0,80.0,1.0,365.0,2020-10,0.0
3,90676,2020-10-27,t,80.0,80.0,1.0,365.0,2020-10,0.0
4,90676,2020-10-28,t,80.0,80.0,1.0,365.0,2020-10,0.0


In [513]:
df.loc[df['month'].isin(['2020-10', '2020-11', '2020-12']) & df['pricedif'] > 0].groupby(['date', 'listing_id']) [['pricedif']].sum().unstack(['date']).fillna(0)

pricedif                        ...                                 
date       2020-10-25 2020-10-26 2020-10-27  ... 2020-12-29 2020-12-30 2020-12-31
listing_id                                   ...                                 
927867            7.0        7.0        7.0  ...        0.0        0.0        0.0
2349920          11.0        0.0        0.0  ...        0.0        0.0        0.0
5820556          10.0       10.0       10.0  ...        0.0        0.0        0.0
12182010         11.0        9.0        0.0  ...        0.0        0.0        0.0
14637285          0.0        0.0        0.0  ...        0.0        0.0        0.0
...               ...        ...        ...  ...        ...        ...        ...
45216400          0.0        5.0        0.0  ...       30.0       30.0       59.0
45231686          5.0        5.0        0.0  ...       30.0       30.0       59.0
45238024          5.0        5.0        0.0  ...       30.0       30.0       59.0
45361245          0.0        0.0        0.0  ...        0.0        0.0       60.0
45974826         22.0       22.0       22.0  ...        0.0        0.0        0.0

[163 rows x 68 columns]

I really struggled with this exercise and ultimately was not able to get the code to run.  I tried resolving errors by using different categorizing variables, creating multiple indexes, and even trying to apply the code to different data sets, but continued to run into new errors or could not solve the original problem.  To produce a result, I decided to create a new column in the data frame that produced the difference in price vs adjusted price I was targeting.  I chose the same months to filter by, grouped by listing_id and date, and unstacked the date.  I also chose to select rows where the pricedif was greater than zero, since there was no difference between prices in the majority of the data.